# UMAP & HDBSCAN Hyperparemeter Tuning

This notebook identifies which parameters for **UMAP** and **HDBSCAN** are optimal for topic modeling with BERTopic. The optimization task has two goals: 

1. Minimize the amount of noise found by HDBSCAN by maximizing the amount of points clustered.
2. Produce confident HDBSCAN clusters by minimizing the number of points with low probability of cluster membership.

In [ ]:
import os
import multiprocessing as mp

import numpy as np
import pandas as pd

from utils.data import load_data
from utils.embeddings import load_embeddings

from umap import UMAP
from hdbscan import HDBSCAN

from functools import partial
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK

from tqdm import tqdm

# set random seed:
np.random.seed(42)

**Load data & embeddings:**

In [ ]:
!ls /datasets/idw-reddit

In [ ]:
DATA = '/datasets/idw-reddit/training_data.csv'
df = load_data(DATA)

EMBEDDING_MODEL = 'all-mpnet-base-v2'
EMBEDDING_MODEL_PATH = os.path.join('embeddings', f'{EMBEDDING_MODEL}.pickle')
embeddings = load_embeddings(EMBEDDING_MODEL_PATH)

assert len(embeddings) == len(df), "Error! Embedding length does not match dataframe length!"

df['embedding'] = list(embeddings)
del embeddings

In [ ]:
df.head()

**Create average embedding representations:**

In [ ]:
# average embeddings:
print('Averaging embeddings')
agg_df = df.groupby(['full_id', 'source'])['embedding'].apply(np.vstack).reset_index()
agg_df['embedding'] = agg_df['embedding'].apply(lambda row: row.mean(axis=0))

# Aggregate sentences & map to embeddings:
print('Mapping aggregated sentences to embeddings')
df = df.groupby(['full_id', 'source'])['tokens'].apply(list).reset_index()
df['tokens'] = df['tokens'].apply(lambda row: ' '.join(row))
agg_df['tokens'] = agg_df['full_id'].map(
    dict(
        zip(
            df['full_id'],
            df['tokens']
        )
    )
)

# save memory
del df

# SORT!
agg_df.sort_values('full_id', ascending=True, inplace=True)

In [ ]:
agg_df.head()

In [ ]:
print(f'Dataframe size: {round(agg_df.memory_usage(deep=True).sum() / 1024**3, 2)} GB')

## Functions

### UMAP & HDBSCAN Fit

- Fit a variety of values for the UMAP & HDBSCAN models.
- Return the fitted HDBSCAN model for analysis.

In [ ]:
def umap_hdbscan_clusterer(umap_embedding: np.array,
                           min_cluster_size: int,
                           min_samples: int) -> HDBSCAN:
    
    # fit HDBSCAN
    hdbscan_model = HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric='euclidean',
        cluster_selection_method='eom',
        prediction_data=True,
        core_dist_n_jobs=mp.cpu_count()-1
    )
    
    clusterer = hdbscan_model.fit(umap_embedding)
    
    return clusterer

### Score Function

- Measure *loss*, defined here as the proportion of points with less than `5%` probability of cluster membership per HDBSCAN.
- Also grab the proportion of points assigned as noise (`-1`).

In [ ]:
def score_func(clusterer: HDBSCAN, threshold: float=0.5) -> tuple:
    labels = clusterer.labels_
    unique_labels = set(sorted(labels))
    
    if -1 in unique_labels:
        n_labels = len(unique_labels) - 1
    else:
        n_labels = len(unique_labels)
    
    probabilites = clusterer.probabilities_
    cost = sum([1 for p in probabilites if p < threshold]) / len(labels)
    noise_proportion = sum([1 for p in labels if p == -1]) / len(labels)
    
    return n_labels, cost, noise_proportion

### Oobjective Function
- Chain `umap_hdbscan_clusterer` and `score_func` together into an objective function.
- Add an additional penalty of `0.15` if the number of clusters returned are less than the lower bound of plausible clusters or higher than the higher bound of plausible clusters.
    - The goal is to avoid HDBSCAN finding a large amount of "micro" clusters that should be part of the same cluster.
    - Additionally, we want to avoid HDBSCAN simply lumping points into an implausibly small number of clusters.

In [ ]:
def objective_function(parameters: dict, umap_embedding: np.array, k_lower: int, k_upper: int) -> dict:
    clusterer = umap_hdbscan_clusterer(
        umap_embedding=umap_embedding,
        min_cluster_size=parameters['min_cluster_size'],
        min_samples=parameters['min_samples'],
    )
    
    k_topics, cost, noise_proportion = score_func(clusterer, threshold=0.05)
    
    # increase penalty for too few and too many clusters:
    if (k_topics < k_lower) | (k_topics > k_upper):
        penalty = 0.15
    else:
        penalty = 0.0
    
    loss = cost + penalty
    eval_dict = {
        'loss': loss, 
        'noise_proportion': noise_proportion, 
        'k_topics': k_topics,
        'min_cluster_size': parameters['min_cluster_size'],
        'min_samples': parameters['min_samples'],
        'status': STATUS_OK
    }
    
    return eval_dict

### Hyperopt Search
- Optimize the search using Hyperopt.
- The *best* parameters are those that most minimize the loss function.

In [ ]:
def hyperopt_search(umap_embedding: np.array,
                    space: dict,
                    k_lower: int,
                    k_upper: int,
                    max_evals: int=100):
    
    trials = Trials()
    
    objective_for_search = partial(
        objective_function,
        umap_embedding=umap_embedding,
        k_lower=k_lower,
        k_upper=k_upper
    )
    
    best_fit = fmin(
        objective_for_search,
        space=space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials=trials
    )
    
    hyperparams = space_eval(space, best_fit)
    print(f'Best fit: {hyperparams}')
    print(f'K topics: {trials.best_trial["result"]["k_topics"]}')
    
    return best_fit, hyperparams, trials
    

## Run the Optimizer

- Establish the search space of UMAP & HDBSCAN parameters.
- Iterate over the search space using Hyperopt.

In [ ]:
umap_model = UMAP(
    n_components=5,
    n_neighbors=15,
    metric='cosine',
    min_dist=0.0,
    init='tswspectral',
    unique=True,
    n_epochs=400,
    low_memory=True,
    random_state=137,
    verbose=True
)

umap_embedding = umap_model.fit_transform(np.array(agg_df['embedding'].tolist()))

In [ ]:
search_space = {
    'min_cluster_size': hp.choice('min_cluster_size', range(50,200+1,1)),
    'min_samples': hp.choice('min_samples', range(2,50+1,1))
}

k_lower = 50
k_upper = 300
max_evals = 100

best_params, hyperparams, trials = hyperopt_search(
    umap_embedding=umap_embedding,
    space=search_space,
    k_lower=k_lower,
    k_upper=k_upper,
    max_evals=max_evals
)

In [ ]:
space_eval(search_space, best_params)

## Evaluate Model Fits
- Iterate over the Hyperopt trials and obtain the performance across parameters.

In [ ]:
model_fits = []
for trial in trials.trials:
    results = trial['result']

    payload = {
        'loss': results['loss'],
        'min_cluster_size': results['min_cluster_size'],
        'min_samples': results['min_samples'],
        'noise_proportion': results['noise_proportion'],
        'k_topics': results['k_topics']
    }

    model_fits.append(payload)

model_fits = pd.DataFrame(model_fits).sort_values('loss', ascending=True).reset_index(drop=True)
model_fits.head(25)

In [ ]:
# save fit data:
print('Saving model fits data')
model_fits.to_csv(
    os.path.join(
        'model_fits',
        'model_fits.csv'
    ),
    index=False
)
print('Done!')